In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from collections import defaultdict

In [2]:
datadir = '/temp_work/ch229121/GlueTraining/setting-12/'
workdir = '/home/ch229121/Projects/Thyme/thyme_model/'

In [3]:
import math

def logit2prob(logit):
  odds = math.exp(logit)
  prob = odds / (1 + odds)
  return prob

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [4]:
def load_probabilities(slurm_id):
    return dict_probabilities[slurm_id]
    
def find_probabilities_for_sampling(probabilities, sampling_no, num_initializations_per_split):
    return probabilities[sampling_no * num_initializations_per_split:
        (sampling_no+1) * num_initializations_per_split]

def find_probabilities_for_optimization(probabilities, sampling_no, num_initializations_per_split):
    return probabilities[sampling_no::num_initializations_per_split]

def load_probabilities_and_get_first_term(slurm_id, num_initializations_per_split, reverse=False):
    '''Variance due to optimization'''
    
    probabilities = load_probabilities(slurm_id)

    num_samplings = probabilities.shape[0] // num_initializations_per_split
    individual_variances = []
    for sampling_no in range(num_samplings):
        if not reverse:
            probabilities_for_this_sampling = find_probabilities_for_sampling(probabilities, sampling_no, num_initializations_per_split)
        else:
            probabilities_for_this_sampling = find_probabilities_for_optimization(probabilities, sampling_no, num_initializations_per_split)
        individual_variance = calculate_variance(probabilities_for_this_sampling)
        individual_variances.append(individual_variance)

    first_term = np.mean(np.array(individual_variances))
    return first_term

def load_probabilities_and_get_second_term(slurm_id, num_initializations_per_split, reverse=False):
    '''Variance due to sampling'''

    probabilities = load_probabilities(slurm_id)

    num_samplings = probabilities.shape[0] // num_initializations_per_split

    expected_probabilities_shape = list(probabilities.shape)
    expected_probabilities_shape[0] = num_samplings

    expected_probabilities = np.zeros(expected_probabilities_shape)

    for sampling_no in range(num_samplings):
        if not reverse:
            probabilities_for_this_sampling = find_probabilities_for_sampling(probabilities, sampling_no, num_initializations_per_split)
        else:
            probabilities_for_this_sampling = find_probabilities_for_optimization(probabilities, sampling_no, num_initializations_per_split)
        expected_probabilities[sampling_no] = np.mean(probabilities_for_this_sampling, 0)

    second_term = calculate_variance(expected_probabilities)

    return second_term

In [5]:
def calculate_variance(bitmaps):
    mean = np.mean(bitmaps, 0)
    return np.mean((bitmaps - np.expand_dims(mean, axis=0)) ** 2)


def calculate_bias(probabilities, test_y_onehot):
    mean = np.mean(probabilities, 0)
    return np.mean((mean - test_y_onehot) ** 2)

def calculate_losses(probabilities, test_y_onehot):
    print(probabilities.shape, test_y_onehot.shape)
    pred = np.argmax(probabilities, axis=1)
    target = np.argmax(test_y_onehot, axis=1)
    losses = np.mean(pred != target, axis=0)
    return losses
    
def load_probabilities_and_get_variances(slurm_id):
    probabilities = load_probabilities(slurm_id)
    original_variance = calculate_variance(probabilities)
    return original_variance

def load_probabilities_and_get_bias(slurm_id):
    probabilities = load_probabilities(slurm_id)
    original_bias = calculate_bias(probabilities, test_y_onehot)
    return original_bias

# Load labels and predictions

In [135]:
from sklearn.preprocessing import OneHotEncoder
labelfile = open("/home/ch229121/Projects/Thyme/thyme_data/test_label.txt", "r")
labels = labelfile.read().splitlines()
labelfile.close()

integer_encoded = np.array(labels)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoder = onehot_encoder.fit_transform(integer_encoded)

/programs/local/anaconda/2019.10/3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [136]:
test_y_onehot_all = onehot_encoder

In [137]:
keep_index = np.where(integer_encoded!='6')[0]

In [138]:
test_y_onehot_no_none = onehot_encoder[keep_index]

In [ ]:
dict_probabilities_all = defaultdict()
# slurm_id = 'PubmedBERTbase-MimicBig-EntityBERT'
# slurm_id = 'roberta-base'
seeds = [42, 52, 62, 72, 82]
splits = ['split_r42', 'split_r52', 'split_r62', 'split_r72', 'split_r82']
for slurm_id in ['PubmedBERTbase-MimicBig-EntityBERT', 'roberta-base']:
    probabilities = []
    for split in splits:
        for seed in seeds:
            df = pd.read_csv(datadir+f'thyme_{slurm_id}_lr.4e-5_linear_epoch.3_seed.{seed}_{split}/predict_results_prob_None.txt', header=0, index_col=0, delimiter='\t')
            df['prediction'] = df.prediction.apply(lambda x: softmax([logit2prob(float(i)) for i in x.split(',')]))
            probabilities.append(np.stack(df.prediction.values))
    probabilities = np.array(probabilities)
    print(probabilities.shape)

    dict_probabilities_all[slurm_id] = probabilities

(25, 208891, 10)

In [ ]:
dict_probabilities_no_none = defaultdict()
for slurm_id in ['PubmedBERTbase-MimicBig-EntityBERT', 'roberta-base']:
    dict_probabilities_no_none[slurm_id] = dict_probabilities_all[slurm_id][:, keep_index]

# Calculate bias and variance

In [139]:
# Global variables
num_initializations_per_split = 5
dict_probabilities = dict_probabilities_no_none
test_y_onehot = test_y_onehot_no_none

In [140]:
slurm_id = 'roberta-base'
bias = load_probabilities_and_get_bias(slurm_id, )
variance = load_probabilities_and_get_variances(slurm_id)
var_opt = load_probabilities_and_get_first_term(slurm_id, num_initializations_per_split)
var_samp = load_probabilities_and_get_second_term(slurm_id, num_initializations_per_split)
print(f'bias: {bias}')
print(f'variance: {variance}')
print(f'variance_optimization: {var_opt}')
print(f'variance_sampling: {var_samp}')
if var_opt > var_samp:
    print('Phase-1')
elif var_opt < var_samp:
    print('Phase-3')
else:
    print('Phase-2')

bias: 0.08128837954355322
variance: 0.000166435612810771
variance_optimization: 0.0001434072005010661
variance_sampling: 2.3028412309705126e-05
Phase-1


In [141]:
slurm_id = 'PubmedBERTbase-MimicBig-EntityBERT'
bias = load_probabilities_and_get_bias(slurm_id, )
variance = load_probabilities_and_get_variances(slurm_id)
var_opt = load_probabilities_and_get_first_term(slurm_id, num_initializations_per_split)
var_samp = load_probabilities_and_get_second_term(slurm_id, num_initializations_per_split)
print(f'bias: {bias}')
print(f'variance: {variance}')
print(f'variance_optimization: {var_opt}')
print(f'variance_sampling: {var_samp}')
if var_opt > var_samp:
    print('Phase-1')
elif var_opt < var_samp:
    print('Phase-3')
else:
    print('Phase-2')

bias: 0.07965660418517094
variance: 8.672195372488624e-05
variance_optimization: 6.512269207249076e-05
variance_sampling: 2.159926165239544e-05
Phase-1


In [112]:
for sampling_no in [0, 1, 2, 3, 4]:
    probabilities_for_this_sampling = dict_probabilities_no_none['roberta-base'][sampling_no::5]
    bias = calculate_bias(probabilities_for_this_sampling, test_y_onehot_no_none)
    var = calculate_variance(probabilities_for_this_sampling)
    print(bias, var)

0.0836389615118786 0.00026890083667728505
0.08139853390668515 0.00015536677771869559
0.08151385272631272 0.00010384289266284722
0.08036358371345685 6.610834769145817e-05
0.07970514960886459 5.977545987179058e-05


In [113]:
for sampling_no in [0, 1, 2, 3, 4]:
    probabilities_for_this_sampling = dict_probabilities_no_none['PubmedBERTbase-MimicBig-EntityBERT'][sampling_no::5]
    bias = calculate_bias(probabilities_for_this_sampling, test_y_onehot_no_none)
    var = calculate_variance(probabilities_for_this_sampling)
    print(bias, var)

0.07945230026220991 7.742348063477951e-05
0.08041242775108372 6.501960345408497e-05
0.07974000258495775 7.146146369743825e-05
0.07930017185001845 6.754454443238603e-05
0.07946107497696271 6.920417702795085e-05


# Code from bias-variance github

In [ ]:
def calculate_variance(bitmaps):
    mean = np.mean(bitmaps, 0)
    return np.mean((bitmaps - np.expand_dims(mean, axis=0)) ** 2)


def get_variance(slurm_id, num_hidden,inter):
    '''
    Returns the variance for a slurm id (corresponding to an experiment) and a hidden size.
    '''
    probabilities = load_probabilities(slurm_id, num_hidden,inter)
    return calculate_variance(probabilities)


def calculate_bias(probabilities, test_y_onehot):
    mean = np.mean(probabilities, 0)
    return np.mean((mean - test_y_onehot) ** 2)


def calculate_losses(probabilities, test_y_onehot):
    print(probabilities.shape, test_y_onehot.shape)
    pred = np.argmax(probabilities, axis=1)
    target = np.argmax(test_y_onehot, axis=1)
    losses = np.mean(pred != target, axis=1)
    return losses


def get_bias(slurm_id, num_hidden, inter):
    '''
    Returns the variance for a slurm id (corresponding to an experiment) and a hidden size.
    '''
    test_y_onehot = get_test_y_onehot()
    probabilities = load_probabilities(slurm_id, num_hidden,inter)
    return calculate_bias(probabilities, test_y_onehot)


def load_probabilities_and_get_variances(slurm_id, hidden_arr,inter=0, num_bootstrap=10000):
    '''
    Loads saved probabilities, calculates differences using bootstrapping from
    the value of variance computed using all the seeds and saves those diffs.
    Prerequisite: Probabilities should be saved earlier using the
    save_probabilities function with dimension (num_seeds, num_test_examples,
    probabilities_for_each_example), eg. (50, 10000, 10) for 50 seeds for MNIST.
    '''
    for num_hidden in hidden_arr:
        probabilities = load_probabilities(slurm_id, num_hidden, inter)
        original_variance = calculate_variance(probabilities)

        diffs = []
        for i in range(num_bootstrap):
            indices = np.random.choice(50, 50, replace=True)
            if slurm_id == 195683:
                indices = np.random.choice(28, 28, replace=True)
            elif slurm_id == 195684:
                indices = np.random.choice(43, 43, replace=True)
            bootstrap_probabilities = probabilities[indices]
            bootstrap_variance = calculate_variance(bootstrap_probabilities)
            diff_variance = (bootstrap_variance - original_variance)
            diffs.append(diff_variance)

        save_variance_diffs(slurm_id, num_hidden, diffs)


def find_probabilities_for_sampling(probabilities, sampling_no, num_initializations_per_split):
    return probabilities[sampling_no * num_initializations_per_split:
        (sampling_no+1) * num_initializations_per_split]

def find_probabilities_for_optimization(probabilities, sampling_no, num_initializations_per_split):
    return probabilities[sampling_no::num_initializations_per_split]

def load_probabilities_and_get_first_term(slurm_id, hidden_arr, num_initializations_per_split, inter=0, reverse=False):
    first_terms = []
    for num_hidden in hidden_arr:
        probabilities = load_probabilities(slurm_id, num_hidden, inter)

        num_samplings = probabilities.shape[0] // num_initializations_per_split
        individual_variances = []
        for sampling_no in range(num_samplings):
            if not reverse:
                probabilities_for_this_sampling = find_probabilities_for_sampling(probabilities, sampling_no, num_initializations_per_split)
            else:
                probabilities_for_this_sampling = find_probabilities_for_optimization(probabilities, sampling_no, num_initializations_per_split)
            individual_variance = calculate_variance(probabilities_for_this_sampling)
            individual_variances.append(individual_variance)

        first_terms.append(np.mean(np.array(individual_variances)))
    return first_terms


def load_probabilities_and_get_second_term(slurm_id, hidden_arr, num_initializations_per_split, inter=0, reverse=False):
    second_terms = []
    for num_hidden in hidden_arr:
        probabilities = load_probabilities(slurm_id, num_hidden, inter)

        num_samplings = probabilities.shape[0] // num_initializations_per_split

        expected_probabilities_shape = list(probabilities.shape)
        expected_probabilities_shape[0] = num_samplings

        expected_probabilities = np.zeros(expected_probabilities_shape)

        for sampling_no in range(num_samplings):
            if not reverse:
                probabilities_for_this_sampling = find_probabilities_for_sampling(probabilities, sampling_no, num_initializations_per_split)
            else:
                probabilities_for_this_sampling = find_probabilities_for_optimization(probabilities, sampling_no, num_initializations_per_split)
            expected_probabilities[sampling_no] = np.mean(probabilities_for_this_sampling, 0)

        second_terms.append(calculate_variance(expected_probabilities))

    return second_terms


def load_probabilities_and_get_biases(slurm_id, hidden_arr, inter =0, num_bootstrap=10000):
    test_y_onehot = get_test_y_onehot()
    for num_hidden in hidden_arr:
        probabilities = load_probabilities(slurm_id, num_hidden, inter)
        original_variance = calculate_bias(probabilities, test_y_onehot)

        diffs = []
        for i in range(num_bootstrap):
            indices = np.random.choice(50, 50, replace=True)
            if slurm_id == 195683:
                indices = np.random.choice(28, 28, replace=True)
            elif slurm_id == 195684:
                indices = np.random.choice(43, 43, replace=True)
            bootstrap_probabilities = probabilities[indices]
            bootstrap_variance = calculate_bias(bootstrap_probabilities, test_y_onehot)
            diff_variance = (bootstrap_variance - original_variance)
            diffs.append(diff_variance)

        save_bias_diffs(slurm_id, num_hidden, diffs)


def load_probabilities_and_get_losses_and_std(slurm_id, hidden_arr,inter =0):
    test_y_onehot = get_test_y_onehot()
    average_losses, stds = [], []
    for num_hidden in hidden_arr:
        probabilities = load_probabilities(slurm_id, num_hidden, inter)
        losses = calculate_losses(probabilities, test_y_onehot)

        average_loss = np.mean(losses)
        std = np.std(losses)/math.sqrt(len(losses))

        average_losses.append(average_loss)
        stds.append(std)

    return average_losses, stds

# Example

In [6]:
test_y_onehot = np.array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.]])
probabilities = np.array([[0.3, 0.6, 0.1], [0.0, 0.8, 0.2], [0.3, 0.7, 0.0], [0.0, 0.0, 1.0]]) 

In [ ]:
def calculate_variance(bitmaps):
    mean = np.mean(bitmaps, 0)
    return np.mean((bitmaps - np.expand_dims(mean, axis=0)) ** 2)


def calculate_bias(probabilities, test_y_onehot):
    mean = np.mean(probabilities, 0)
    return np.mean((mean - test_y_onehot) ** 2)

In [9]:
(np.mean(probabilities, 0) - test_y_onehot) ** 2

array([[0.7225  , 0.275625, 0.105625],
       [0.0225  , 0.225625, 0.105625],
       [0.0225  , 0.225625, 0.105625],
       [0.0225  , 0.225625, 0.105625]])

In [10]:
np.mean((np.mean(probabilities, 0) - test_y_onehot) ** 2)

0.18041666666666667

In [5]:
var = calculate_variance(probabilities)
bias = calculate_bias(probabilities, test_y_onehot)
print(var, bias)

0.09208333333333334 0.18041666666666667


In [14]:
import torch
MNIST_TEST_SIZE = 4
NUM_MNIST_CLASSES = 3
test_y_onehot = torch.FloatTensor(MNIST_TEST_SIZE, NUM_MNIST_CLASSES)
print(test_y_onehot)
test_y_onehot.zero_()
print(test_y_onehot)
index = torch.tensor([[0, 1, 2], [0, 0, 0], [0, 1, 2], [0, 1, 0]])
test_y_onehot.scatter_(1, index, 1)
print(test_y_onehot)
test_y_onehot = test_y_onehot.cpu().numpy()
print(test_y_onehot)

tensor([[1.0693e-05, 3.1369e+27, 7.0800e+31],
        [3.1095e-18, 1.8590e+34, 7.7767e+31],
        [7.1536e+22, 3.3803e-18, 1.9421e+31],
        [2.7491e+20, 6.1949e-04, 1.3527e-08]])
tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
tensor([[1., 1., 1.],
        [1., 0., 0.],
        [1., 1., 1.],
        [1., 1., 0.]])
[[1. 1. 1.]
 [1. 0. 0.]
 [1. 1. 1.]
 [1. 1. 0.]]
